# Initializer

* -hu how are you?
* -hd how is your day?

* An adverb is a part of speech used to describe a verb, adjective, clause, or another adverb. It simply tells the readers how, where, when, or the degree at which something was done. Examples: The manager accepted the challenge very nicely. The manager accepted the challenge very nicely.

In [1]:
dataset = [
    { "tag":"hi", "pattern": [ 'hi', 'hello', 'yo','hey','greetings','good morning','evening',"sup",'afternoon' ], 
     "response":['Hi, how is your day? -hd', 'Hello, how are you? -hu'], "state": False},
    {"tag": "state_of_being","pattern": ["hi,How are you?", "how are you", "Whats up", "how are you doing this morning?", "doing this afternoon?", "this evening?", "how far?", "this morning", "doing this evening?"], 
     "response": [], "state": False},
    {"tag": "name","pattern": ["What is your name?", "can I know you", "your name please","what should I call you", "who are you", "what are you called?", "can I know your name", "please I wish to know your name?"],
     "response": [], 'state': False},
    {"tag": "create_you","pattern": ["who created you", "who made you", "who built you", "who developed you", "who is your founder"],
     "responses": ["I was made by a team at AppsTechLabs", "I was made by a team of people, at AppsTechLabs", "A team at AppsTechLabs made me what I am today, the are sort of like my family, they always ensure I bring more value to you. trying saying, help, to see a list of what I can do"],
    "state": False},
    {"tag": "owner", "pattern": ["who is your owner", "woner", "who owns you"],"responses": ["I think I'm yours", "I'm yours", "guess that would be you", "I'm pretty sure that would be you"],"state": False},
    {"tag": "know me","pattern": ["do you know me?", "do you know my name?"],"responses": ["Sorry that I don't know you for now, but trust me it won't be long. (:-"], "state": False},
    {"tag": "age", "pattern": ["how old are you"," whats your age","tell me your age","can I know how old you are?","can I know your age"], 
     "responses": ["I don't age, but I was made by a team at AppsTechLabs. They made me what I am today."],'state': False},
    {"tag": "creator","pattern": ["what's your creator name", "who is your creator", "whats your creator name", "what is your creator name", "creator", "who program you"],
    "responses": ["At first, I was just an idea, then a bunch of people at AppsTechLabs put their heads together, And now, here I am", "Everyone at AppsTechLabs is sort of like my family, they made me what I am today",
                                "I consider everyone at AppsTechLabs to be my family, it's a lot of birthdays to remember, but I like it",
                                "The AppsTechLabs team is like my family, they mean a lot to me",
                                "AppsTechLabs is my family"], 'state': False }
]

# Rule-Base Matching And Spell Check

In [7]:
def uppercase_decorator(function):
    def wrapper():
        func = function()
        make_uppercase = func.upper()
        return make_uppercase

    return wrapper

def split_string(function):
    def wrapper():
        func = function()
        splitted_string = func.split()
        return splitted_string

    return wrapper


def say_hi():
    return 'hello there'


print(say_hi())

AttributeError: 'list' object has no attribute 'upper'

In [2]:
import spacy
from textblob import TextBlob
import nltk

spacy.prefer_gpu()
sp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
m_tool = Matcher(sp.vocab)


# Patterns
hi_hu = [{"LOWER": "-hu"}]
hi_hd = [{"LOWER": '-hd'}]

# Add patterns to matcher object
m_tool.add('HI', None, hi_hd, hi_hu)

# 1
def matcher(sent):
    sentence = sp(sent)
    phrase_match = m_tool(sentence)
    
    for match_id, start, end in phrase_match:
        string_id = sp.vocab.strings[match_id]
        span = sentence[start: end]
        
        return string_id, span.text, sent.replace(f' {span.text}', '')
# 3
def Hi(arg):
    if len(arg) == 4:
        tag, pattern, model_res, user_res = arg
        data = {'tag':tag, 'pattern': pattern, 'model_res': model_res, 'user_res': user_res}
        return data
    
    elif len(arg) == 3:
        tag, pattern, model_res = arg
        data = {'tag':tag, 'pattern': pattern, 'model_res': model_res, 'user_res': ''}
        return data
# 2
def tag_matcher(sent):
    arg = matcher(sent)
    tag, pattern, model_res = arg
    
    selector = {
        "HI": Hi
    }
    
    return selector.get(tag)([tag,pattern,model_res])

def how_are_you(sent):
    sentence = nltk.word_tokenize(sent)
    sentence = nltk.pos_tag(sentence)
    sent_in_pos = ''
    
    for word in sentence:
        sent_in_pos += word[1] + ' '
    if 'JJ CC PRP' in sent_in_pos or 'CC PRP' in sent_in_pos:
        res = [ tag for tag in sentence if "JJ" in tag[1]]
        print(sent, sent_in_pos)
        return True, res[0]
    elif 'JJ' in sent_in_pos or 'NN' in sent_in_pos:
        res = [ tag for tag in sentence if "JJ" in tag[1] or "NN" in tag[1]]
        return False, res[0]
        

def pos_tag(sent):
    text_blob_object = TextBlob(sent).correct()
    state, res = how_are_you(str(text_blob_object))
#     print(res, state)
    if state:
        if text_blob_object.sentiment.polarity > 0:
            res = f"Bot: I'm glad you are {res[0]}. \nBot: As for me, I'm awesome. Thanks for asking. \nBot: Please I wish to know your name."
            return 'Pos', res
        else:
            res = f"Bot: I'm sorry about that, is there anything I can do to make you smile?"
            return 'Neg',res
    else:
        if text_blob_object.sentiment.polarity > 0:
            res = f"Bot: I'm glad you are {res[0]}. \nBot: What's on your mind for today?"
            return 'Pos', res
        else:
            res = f"Bot: I'm sorry about that, is there anything I can do to make you smile?"
            return 'Neg',res

In [3]:
print(pos_tag("I am not too awesome"))

('Pos', "Bot: I'm glad you are awesome. \nBot: What's on your mind for today?")


In [4]:
import nltk

text = nltk.word_tokenize("following day")
hey = nltk.pos_tag(text)
for me in hey:
    print(me)
    if me[1] in ('JJ', 'CC'):
        print(me)

('following', 'VBG')
('day', 'NN')


In [5]:
import nltk

h = ['set an event', 'set up an event', 'book an event', 'book event','hook an event',
    'hook up an event']
for sen in h:
    text = nltk.word_tokenize(sen)
    hey = nltk.pos_tag(text)
    print()
    for me in hey:
        print(me)


('set', 'VB')
('an', 'DT')
('event', 'NN')

('set', 'VBN')
('up', 'RP')
('an', 'DT')
('event', 'NN')

('book', 'NN')
('an', 'DT')
('event', 'NN')

('book', 'NN')
('event', 'NN')

('hook', 'NN')
('an', 'DT')
('event', 'NN')

('hook', 'VB')
('up', 'RP')
('an', 'DT')
('event', 'NN')


In [6]:
import spacy
sp = spacy.load('en_core_web_sm')

h = ['set an event', 'set up an event', 'book an event', 'book event', 'hook an event',
    'hook up an event']
for letter in h:
    sen = sp(letter)
    print()
    for word in sen:
        print(f'{word.text:{12}} {word.pos_:{10}} {word.tag_:{8}} {spacy.explain(word.tag_)}')


set          VERB       VB       verb, base form
an           DET        DT       determiner
event        NOUN       NN       noun, singular or mass

set          VERB       VB       verb, base form
up           ADP        RP       adverb, particle
an           DET        DT       determiner
event        NOUN       NN       noun, singular or mass

book         PROPN      NNP      noun, proper singular
an           DET        DT       determiner
event        NOUN       NN       noun, singular or mass

book         NOUN       NN       noun, singular or mass
event        NOUN       NN       noun, singular or mass

hook         VERB       VB       verb, base form
an           DET        DT       determiner
event        NOUN       NN       noun, singular or mass

hook         VERB       VB       verb, base form
up           ADP        RP       adverb, particle
an           DET        DT       determiner
event        NOUN       NN       noun, singular or mass


In [7]:
sens = sp("9:30pm is the time is ")
h =  [ (ent.label_, ent.text) for ent in sens.ents if ent.label_ is 'DATE' or ent.label_ is 'TIME']
if len(h) is not 0:
    for ent in h:
        print(ent[0])

for entity in sens.ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

9:30pm - CARDINAL - Numerals that do not fall under another type


In [13]:
message = "set up and event the following day by 4 pm in the evening"

def event_date(message):
    sentence = sp(message)
    sentence = [ (ent.label_, ent.text) for ent in sentence.ents if ent.label_ is 'DATE' or ent.label_ is 'TIME']
    
    if len(sentence) is not 0:
        for ent in sentence:
            if ent[0] in 'DATE':
                date = ent[1]
                num = ''.join([i for i in date if i.isdigit()])
                if num:
                    print(next_some_days(int(num)))
                else:
                    time = {
                        'today': today,
                        'tomorrow': tomorrow
                    }
                    
                    print(time.get(date))
            elif ent[0] in 'TIME':
                time = ent[1]
                num = ''.join([i for i in time if i.isdigit()])
                if 'am' in time.lower() :
                    print("AM", time, num)
                elif 'pm' in time.lower():
                    print('PM', time, num)
                
                
event_date(message)

None
PM 4 pm 4


In [9]:
from datetime import datetime, timedelta

date = datetime.now()
next_day = date + timedelta(days = 1)

today = date.strftime("%B %-d, %Y")
tomorrow = next_day.strftime("%B %-d, %Y")

def next_some_days(num):
    date_ = date + timedelta(days = num)
    return date_.strftime("%B %-d, %Y")

print(today, tomorrow)

March 31, 2021 April 1, 2021


In [10]:
from datetime import datetime
# now = datetime.fromtimestamp(1585892034661 / 1000.0)

now = datetime.now()

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%m/%d/%Y, %H:%M:%S %p")
print("date and time:",date_time)	

print(datetime.today(), 'love')

year: 2021
month: 03
day: 31
time: 08:37:02
date and time: 03/31/2021, 08:37:02 AM
2021-03-31 08:37:02.338861 love


In [11]:
from spacy import displacy

sen = sp(u"I like to play football. I hated it in my childhood though")
displacy.render(sen, style='dep', jupyter=True, options={'distance': 85})

In [14]:
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import tflearn
import tensorflow as tf
from mute_tf_warnings import tf_mute_warning
import random
from time import sleep

stemmer = LancasterStemmer()
tf_mute_warning()

class ChatBot:
        
    
    def data_mining(self):
        data = dataset
        words = [] # we tokenize all the patterns on our data
        labels = [] # get all tags
        docs_x = [] # pattern without being tokenize
        docs_y = [] # labels for pattern without being tokenize


        for tag in data:
            for pattern in tag['pattern']:
                wrds = word_tokenize(pattern)
                words.extend(wrds)
                docs_x.append(wrds)
                docs_y.append(tag['tag'])

            if tag['tag'] not in labels:
                labels.append(tag['tag'])
                
        # Stemming and Remove Duplicates
        words = [ stemmer.stem( word.lower() ) for word in words if word not in '?' and word not in '.']
        words = list(dict.fromkeys(words))
        labels = sorted(list(set(labels)))
        
        # Training Data
        training = []
        labels_for_training = []

        for index, word in enumerate(docs_x):
            temp = []
            wrds = [stemmer.stem(w.lower()) for w in word] # loop through each pattern and stem the the word

            for w in words: # comparing with our tokenize pattern
                if w in wrds:
                    temp.append(1)
                else:
                    temp.append(0)

            output_row = [ 0 for _ in range(len(labels)) ]
            output_row[labels.index(docs_y[index])] = 1

            training.append(temp)
            labels_for_training.append(output_row)

        training = np.array(training)
        labels_for_training = np.array(labels_for_training)

        return training, labels_for_training, words, labels
    
    def test_model(self, test_statement):
        _,_,words,_ = self.data_mining()
        bunch_of_words = [ 0 for _ in range(len(words))]
        
        test_input = word_tokenize(test_statement)
        test_input = [ stemmer.stem(word.lower()) for word in test_input ]
        
        for test in test_input:
            for index, word in enumerate(words):
                if word == test:
                    bunch_of_words[index] = 1
                    
        return np.array(bunch_of_words)
        
        
    # Model
    def model(self):
        training, labels_for_training, _, _= self.data_mining()
        
        tf.reset_default_graph()
        net = tflearn.input_data(shape=[None, len(training[0])])
        net = tflearn.fully_connected(net, 8)
        net = tflearn.fully_connected(net, 8)
        net = tflearn.fully_connected(net, len(labels_for_training[0]), activation="softmax")
        net = tflearn.regression(net)

        model = tflearn.DNN(net)
        
        return model
    
    def test_and_train_model(self):
        training, labels_for_training, _, labels = self.data_mining()
        model = self.model()
        
        model.fit(training, labels_for_training, n_epoch=150,batch_size=6, show_metric=True)
        
        test_data = [ {"tag": tag['tag'], "pattern": tag['pattern'][0]} for tag in dataset ]
        
        for pattern in test_data:
            tag = pattern['tag']
            result = model.predict([self.test_model(pattern['pattern'])])
            result_index = np.argmax(result)
            result_tag = labels[result_index]

            if tag == result_tag:
                print(True,result_tag,':\t',tag,)
            else:
                print(False, result_tag,':\t',tag,':/t',pattern['pattern'])
                self.test_and_train_model()
                break
        model.save('./saved models/model.tflearn')
        
    def user_input(self, user_text):
        _, _, words, _ = self.data_mining()
        word_que = [ 0 for _ in range(len(words))]

        token_words = word_tokenize(user_text)
        token_words = [ stemmer.stem( word.lower() ) for word in token_words ]

        for word in token_words:
            for index, data in enumerate(words):
                if data == word:
                    word_que[index] = 1
        return np.array(word_que)


    def talk(self, txt):
        _,_,_, labels = self.data_mining()
        model = self.model()
        try:
            model.load('./saved models/model.tflearn')
            model_result = model.predict([self.user_input(txt)])
            model_result_index = np.argmax(model_result)
            
            tag = labels[model_result_index]
            
            response = self.fetch_tag_and_response(tag)
            
            if model_result[0][model_result_index] >= 0.8:
                res_tag = response['tag']
                responses = response['responses']
                
                print(random.choice(responses))
            else:
                print('Try asking something else') # in case of this, it should deviate user from conversation and suggest new tags
#             print(model_result[0][model_result_index], response['tag'])
        except:
            self.test_and_train_model()
            sleep(3)
            self.talk()
            
            
    def test_me(self):
        model = self.model()
        _, _, _, labels = self.data_mining()
        quest = False
        database = []
        try:
            model.load('./saved models/model.tflearn')
            print('Welcome \n')
            
            while True:
                user_text = input('You: ')
                if user_text.lower() == 'stop':
                    break
                
                if quest:
                    database[0]['user_res'] = user_text
#                     print(database)
                    pos_, model_res = pos_tag(user_text)
                    sleep(1)
                    print(model_res)
                    quest = False

                else:
                    result = model.predict([self.user_input(user_text)])[0]
                    result_index = np.argmax(result)
                    tag = labels[result_index]

    #                 _tag = [ data for data in dataset if data['tag'] == tag]
    #                 _res = _tag[0]['response']

                    for data in dataset:
                        if data['tag'] == tag:
                            if len(data['response']) != 0 and data['state'] != True:
                                _res = data['response'].pop()

                                matcher_res = tag_matcher(_res)
                                database.append(matcher_res)
                                quest = True

                                print(tag_matcher(_res),database)
                                sleep(1)
                                print('Bot: ',matcher_res['model_res'])

                                break
    #                 print(tag,'\n',result)
        except:
            print('Save Model Not Found')
            print('Initializing Traing and Testing \n')
#             self.test_and_train_model()
#             sleep(3)
             
#             print('Training Successful')
#             print('Initializing Chat')
#             sleep(4)
#             self.test_me()
            
new_bot = ChatBot()
new_bot.test_and_train_model()

ModuleNotFoundError: No module named 'tflearn'

In [6]:
new_bot.test_me()

NameError: name 'new_bot' is not defined

In [7]:
new_bot.test_me()

NameError: name 'new_bot' is not defined

In [ ]:
new_bot.test_me()

In [ ]:
new_bot.test_me()

In [ ]:
new_bot.test_me()

In [8]:
pip install mute_tf_warnings

Note: you may need to restart the kernel to use updated packages.


In [16]:
def filter(filter_by:list, filter: list):
    print(item= for item in filter_by)

In [17]:
filter([1,2], [4,8])

1 2 3 4


In [21]:
class User:
    def __init__(self, username: str, id: int):
        self.username = username
        self.id = id
        
    @staticmethod
    def get_instance(obj: dict):
        return User(**obj)
    
user = User.get_instance({"username": "raymond", "id": 20})

In [27]:
def hey(a:list(dict)):
    print(a)


TypeError: 'type' object is not iterable